In [1]:
import time
import random

import pandas as pd
import numpy as np


from faker import Factory

In [2]:
fake = Factory.create()

In [3]:
companies = pd.read_csv('label.csv')
companies

,label,name
0,Beauty,Bobbi Brown Inc.
1,Beauty,Dior Beauty
2,Beauty,Derma Skincare Centre
3,Beauty,KIKO Asia Ltd.
4,Beauty,Vichy Laboratories
5,Beauty,Colourmix Cosmetics Company Limited
6,Beauty,Sa Sa Cosmetic Co. Ltd.
7,Beauty,"Clinique Laboratories, llc."
8,Beauty,"Shiseido Company, Limited"
9,Beauty,"Sephora USA, Inc."


In [9]:
#companies = pd.Series(companies.name.values)


In [8]:
# random.shuffle(companies)


In [8]:
def generate_people(n):
    """ Generates n-people, and returns a data frame.
        Each person has a:
            - id
            - age
            - name
            - address
            
        Two of them may be unexpected:
            - compromised: 0 or 1
            - compromised: if compromised, when
            
        These values get set by other functions."""
#     namePool = np.load('names.npy')
#     ids = [i for i in range(n)]
#     names = [fake.name() for n in range(n)]
#     uuids = [fake.uuid4() for n in range(n)]
#     ages = [random.randint(18, 80) for n in range(n)]
#     addresses = [fake.address().replace('\n', ', ') for n in range(n)]
#     compromised = list(np.zeros(n))
#     compromised_time = list(np.zeros(n))
    #generate names
    names = set()
    while len(names) < n:
        nextName = fake.name()
        if nextName not in names:
            names.add(nextName)
    names = list(names)
    names = np.array(names)
    genders = np.random.choice(['male', 'female'], n)
    ids = np.arange(n)
    ages = [random.randint(18, 50) for _ in range(n)]
    addresses = [fake.address().replace('\n', ', ') for _ in range(n)]
    compromised = list(np.zeros(n))
    compromised_time = list(np.zeros(n))
    df = pd.DataFrame([ids, names, genders, ages, addresses, compromised, compromised_time]).T
    df.columns = ['id', 'name', 'gender', 'age', 'address', 'compromised', 'compromised_time']
    return df



In [12]:
def generate_business(n, fraudster_likelihood=0.01):
    """ Generates n-businesses, and returns a data frame.
        Each person has a:
            - id
            - name
            - address
            
        The one weird one is "frauders_present" which is controlled by the fraudster_likelihood option.
            
        These values get set by other functions."""
    
    ids = [i for i in range(n)]
#     names = list(companies[:n])
    coms = companies.sample(n, replace=True)
    addresses = [fake.address().replace('\n', ', ') for n in range(n)]
    fraudsters = [1 if random.random() < fraudster_likelihood else 0 for n in range(n) ]
    
    df = pd.DataFrame([ids, list(coms['name']), list(coms['label']), addresses]).T
    df.columns = ['id', 'name', 'label', 'address']
    return df


In [24]:
def generate_normal_transactions(people, businesses, max_transactions=40, fraud_likelihood=1, earliest="-30d", latest="now", amin=9, amax=300):
    """ Generates normal transactions. Returns a list."""
    transactions = []
    amounts = []
    companyNameList = []
    companyLabelList = []
    times = []
    users = []
    ageList = []
    genderList = []
    # For each person
    for p in range(len(people)):
        
        person = people.iloc[p]
        
#         number_of_transactions = random.randint(1, max_transactions)
        number_of_transactions= max_transactions
        # Generate soem transactions
        for i in range(number_of_transactions):

            # random time and business
            time = fake.date_time_between(start_date=earliest, end_date=latest, tzinfo=None)
#             time = fake.date_time_this_month()
            business = businesses.sample(1)
            companyName = business['name']
            companyLabel = business['label']
            amount = random.randint(amin, amax) + round(random.random(), 2)

            
            times.append(time)
            companyNameList.append(companyName)
            companyLabelList.append(companyLabel)
            amounts.append(amount)
            users.append(person['name'])
            ageList.append(person.age)
            genderList.append(person.gender)
#             transactions.append(transaction)
    companyList= companies.sample(len(times), replace=True)
    companyNameList = list(companyList['name'])
    companyLabelList = list(companyList['label'])
    
    df = pd.DataFrame([times, users, genderList, ageList, companyNameList, companyLabelList, amounts]).T
    df.columns = ['time', 'name', 'gender' ,'age', 'company name', 'company label', 'amount']
    return df
# generate_normal_transactions(p,b)


In [9]:
# def generate_fradulent_transactions(people, businesses, max_transactions=40, user_fraud_detection_likelihood=0.10, latest="+10d", amin=1, amax=3000):
#     """ Genereates fradulent transactions. Returns a list."""
#     transactions = []
#     compromised = people[people.compromised == 1]
    
#     for i in range(len(compromised)):
        
#         person = compromised.iloc[i]
#         earliest = person.compromised_time
#         time = fake.date_time_between(start_date=earliest, end_date=latest, tzinfo=None)
# #         number_of_transactions = random.randint(5, max_transactions)
#         number_of_transactions = max_transactions
#         for j in range(number_of_transactions):
    
#             def fraud():        

#                 business = businesses.sample(1)
#                 amount = random.randint(amin, amax) + round(random.random(), 2)

#                 transaction = '{time} uuid={uuid} user="{user}" business="{business}" amount={amount} disputed=true'.format(
#                     time = time,
#                     uuid = person['uuid'],
#                     user = person['name'],
#                     business = business.name.values[0],
#                     amount = amount)

#                 transactions.append(transaction)

#             if j == 1:
#                 fraud()

#             # Each fradulent transaction has +1% chance of being cause from the user
#             # Imaginging they are maybe monitoring their transaction history
#             # Or happen to be checking one thing or another
#             if random.random() < j/100:
#                 fraud()
            
#     return transactions

In [10]:
generate_people(3)

,id,name,age,address,compromised,compromised_time
0,0,Erin Burton,51,"00076 Warren Inlet, North Justinchester, IN 97410",0,0
1,1,Robert Bush,56,"528 Duran Port, Ericland, TX 14494-3816",0,0
2,2,Scott Hansen,63,"49906 Rivera Rest Suite 699, East Daniel, AS 5...",0,0


In [12]:
generate_business(3)

AttributeError: 'DataFrame' object has no attribute 'name'

In [17]:
p = generate_people(10000)
b = generate_business(3000)
b

,id,name,label,address
0,0,New Oriental Education & Tech Grp,Education,"1075 Santiago Gateway Apt. 840, South John, NM..."
1,1,Din Tai Fung,Dining,"6004 Rachel Points, West Nicolemouth, OK 36734..."
2,2,Nike,Clothing,"25100 Martinez Forks, Port Christine, IN 13287..."
3,3,"The 'Star' Ferry Company, Limited",Transport,"545 Daniels Parkway Suite 779, Lake Tanyaberg,..."
4,4,"2U, Inc.",Education,"PSC 1593, Box 8631, APO AA 40984-1758"
5,5,"The 'Star' Ferry Company, Limited",Transport,"3224 Anderson Turnpike, Lake Luis, WV 45334-6452"
6,6,"Vudu, Inc.",Media,"814 Diana Springs Apt. 930, North Jacobhaven, ..."
7,7,China Distance Education Hldgs Ltd,Education,"7983 Aaron Cliff Suite 831, Nelsonport, OK 32924"
8,8,AMC Entertainment Inc,Entertainment,"Unit 9786 Box 6665, DPO AA 70194"
9,9,City Super Limited,Grocery,"81893 Cheryl Circle, Kathrynport, NY 54107"


In [25]:
transactionDf = generate_normal_transactions(p,b)
transactionDf

,time,name,gender,age,company name,company label,amount
0,2017-07-02 22:27:48,Rachel Trujillo,male,36,Coursera Inc.,Education,111.22
1,2017-06-10 12:37:05,Rachel Trujillo,male,36,Laura Mercier Cosmetics.,Beauty,108.46
2,2017-06-07 14:24:02,Rachel Trujillo,male,36,Synovus Financial Corp.,Investment,208.21
3,2017-06-25 15:36:22,Rachel Trujillo,male,36,Peak Tramways Company Limited,Transport,222.79
4,2017-06-30 03:34:48,Rachel Trujillo,male,36,McDonald' s,Dining,86.28
5,2017-06-29 18:00:00,Rachel Trujillo,male,36,Pearson plc,Education,47.68
6,2017-06-30 21:34:26,Rachel Trujillo,male,36,Wal-Mart Stores Inc,Grocery,93.69
7,2017-07-04 11:57:56,Rachel Trujillo,male,36,Principal Financial Group Inc,Investment,115.46
8,2017-06-13 06:54:58,Rachel Trujillo,male,36,"Voya Financial, Inc.",Investment,142.11
9,2017-07-02 18:23:56,Rachel Trujillo,male,36,"The Dairy Farm Company, Limited",Grocery,61.87


In [26]:
transactionDf.loc[transactionDf['name']=='Kelsey Jones', 'name'].shape

(40,)

In [27]:
np.mean(list(transactionDf.age))
# transactionDf = transactionDf[['time','name','','age','company name', 'company label', 'amount']]
transactionDf.to_csv('transaction.csv', index=False)

In [28]:
df = pd.read_csv('transaction.csv')
df

,time,name,gender,age,company name,company label,amount
0,2017-07-02 22:27:48,Rachel Trujillo,male,36,Coursera Inc.,Education,111.22
1,2017-06-10 12:37:05,Rachel Trujillo,male,36,Laura Mercier Cosmetics.,Beauty,108.46
2,2017-06-07 14:24:02,Rachel Trujillo,male,36,Synovus Financial Corp.,Investment,208.21
3,2017-06-25 15:36:22,Rachel Trujillo,male,36,Peak Tramways Company Limited,Transport,222.79
4,2017-06-30 03:34:48,Rachel Trujillo,male,36,McDonald' s,Dining,86.28
5,2017-06-29 18:00:00,Rachel Trujillo,male,36,Pearson plc,Education,47.68
6,2017-06-30 21:34:26,Rachel Trujillo,male,36,Wal-Mart Stores Inc,Grocery,93.69
7,2017-07-04 11:57:56,Rachel Trujillo,male,36,Principal Financial Group Inc,Investment,115.46
8,2017-06-13 06:54:58,Rachel Trujillo,male,36,"Voya Financial, Inc.",Investment,142.11
9,2017-07-02 18:23:56,Rachel Trujillo,male,36,"The Dairy Farm Company, Limited",Grocery,61.87


In [29]:
df['time']

0         2017-07-02 22:27:48
1         2017-06-10 12:37:05
2         2017-06-07 14:24:02
3         2017-06-25 15:36:22
4         2017-06-30 03:34:48
5         2017-06-29 18:00:00
6         2017-06-30 21:34:26
7         2017-07-04 11:57:56
8         2017-06-13 06:54:58
9         2017-07-02 18:23:56
10        2017-06-17 03:32:26
11        2017-07-03 13:44:19
12        2017-06-16 16:16:17
13        2017-06-17 01:17:02
14        2017-06-21 11:14:37
15        2017-06-11 21:15:08
16        2017-06-14 11:45:03
17        2017-07-03 21:29:17
18        2017-06-20 09:04:47
19        2017-06-10 18:31:22
20        2017-07-05 20:58:23
21        2017-06-23 20:15:43
22        2017-06-24 15:32:14
23        2017-06-24 04:00:43
24        2017-07-04 03:13:19
25        2017-06-12 00:30:44
26        2017-06-20 09:50:33
27        2017-06-22 22:08:41
28        2017-06-11 13:26:35
29        2017-07-04 09:40:59
                 ...         
399970    2017-06-12 23:41:20
399971    2017-06-29 22:55:52
399972    

In [19]:
t.extend(f)
len(t)

1927

In [20]:
with open('output.log', 'w') as f:
    for line in t:
        f.write(line)
        f.write('\n')

In [21]:
! ls

LICENSE                                credit-card-fraud-data-generator.ipynb
README.md                              output.log
companies.csv                          requirements.txt


In [22]:
! head output.log

2017-07-02 12:17:52 uuid=6a8d3fa6-e47e-4c59-a469-c2a4a4ac4247 user="Lisa Henderson" business="Emergent Biosolutions, Inc." amount=267.69 disputed=false
2017-07-02 12:18:03 uuid=6a8d3fa6-e47e-4c59-a469-c2a4a4ac4247 user="Lisa Henderson" business="Blackrock Core Bond Trust" amount=245.13 disputed=false
2017-07-02 12:18:03 uuid=6a8d3fa6-e47e-4c59-a469-c2a4a4ac4247 user="Lisa Henderson" business="Phillips 66 Partners LP" amount=160.64 disputed=false
2017-07-02 12:17:40 uuid=6a8d3fa6-e47e-4c59-a469-c2a4a4ac4247 user="Lisa Henderson" business="Kimberly-Clark Corporation" amount=148.37 disputed=false
2017-07-02 12:17:37 uuid=6a8d3fa6-e47e-4c59-a469-c2a4a4ac4247 user="Lisa Henderson" business="Seaspan Corporation" amount=233.69 disputed=false
2017-07-02 12:17:56 uuid=6a8d3fa6-e47e-4c59-a469-c2a4a4ac4247 user="Lisa Henderson" business="Time Warner Inc." amount=164.46 disputed=false
2017-07-02 12:18:13 uuid=6a8d3fa6-e47e-4c59-a469-c2a4a4ac4247 user="Lisa Henderson" business="Flaherty & Crumrine 